In [10]:
import cv2 as cv
import cv2.aruco as aruco 
import numpy as np 

calib_data_path = r"C:\Users\Crist\OneDrive\Desktop\COMPUTER VISION\AUGMENTED_REALITY\calib_data\MultiMatrix.npz"
calib_data = np.load(calib_data_path)
print(calib_data.files)

cam_mat = calib_data["camMatrix"]
dist_coef = calib_data["distCoef"]
r_vector = calib_data["rVector"]
t_vector = calib_data["tVector"]

MARKER_SIZE = 3.7  # centimeters

['camMatrix', 'distCoef', 'rVector', 'tVector']


In [ ]:
marker_dict = aruco.getPredefinedDictionary(cv.aruco.DICT_4X4_50)
detector_params = cv.aruco.DetectorParameters()

cap = cv.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        print("Impossibile leggere frame ")
        break
    gray_frame = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    detector = cv.aruco.ArucoDetector(marker_dict, detector_params)
    marker_corners, marker_ids, rejected_candidates = detector.detectMarkers(gray_frame)
    
    if marker_corners:
        rVec, tVec, _ = cv.aruco.estimatePoseSingleMarkers(marker_corners, MARKER_SIZE, cam_mat, dist_coef)
        # marker_corners: first, second, third, fourth corners
        total_markers = range(0, marker_ids.size)
        for ids, corners, i in zip(marker_ids, marker_corners, total_markers):
            cv.polylines(frame, [corners.astype(np.int32)], True, (0,255,255), 4, cv.LINE_AA)
            corners = corners.reshape(4,2)
            corners = corners.astype(int)
            top_right = corners[0].ravel()
            top_left = corners[1].ravel()
            bottom_right = corners[2].ravel()
            bottom_left = corners[3].ravel()
            # Draw the pose of the marker
            point = cv.drawFrameAxes(frame, cam_mat, dist_coef, rVec[i], tVec[i], 3, 2)
            cv.putText(frame, f"id: {ids[0]} Dist: {round(tVec[i][0][2])},2", top_right, cv.FONT_HERSHEY_PLAIN, 1.3, (200,100,255), 2, cv.LINE_AA)
            cv.putText(frame, f"X: {round(tVec[i][0][0])} Y: {round(tVec[i][0][1])},2", bottom_left, cv.FONT_HERSHEY_PLAIN, 1.3, (0,100,200), 2, cv.LINE_AA)
            print(ids, "  ", corners )
    cv.imshow("frame", frame)
    key = cv.waitKey(1)
    if key == ord('q'):
        break

cap.release()
cv.destroyAllWindows()